# Introduction to NLP with Python spaCy: analyzing restaurant reviews


_This work is licensed under a [Creative Commons BY-SA 4.0 License](http://creativecommons.org/licenses/by-sa/4.0/)_

<br><br><br><br>
__Daniel Kapitan__<br>
`e. d.kapitan@jads.nl`<br>
`w. https://kapitan.net`<br>



<img style="float: left" src="https://github.com/jads-nl/public-lectures/blob/main/logos/jads-gold-250x60.png?raw=true">

## The challenge: predict the restaurant review score with the review text

Thanks to [the people at analyticslab.nl](https://www.theanalyticslab.nl/about-us/) we will use a restaurant review dataset as our introduction to NLP with spaCy. The dataset contains nearly 370.000 reviews collected over an eight-year period, which they scraped from the original review website (I think it is iens.nl). We will follow along [their blogpost series](https://www.theanalyticslab.nl/nlpblogs_0_preparing_restaurant_review_data_for_nlp_and_predictive_modeling/), but replacing their R code with a workflow in Python spaCy.

In this notebook we compare different NLP techniques to show you how we get valuable information from unstructured text. Given the restaurant reviews, the challenge is whether these could be used to:
- Extract more structured information from the review texts without them having to read all the reviews one by one
- Extract actionable feedback to the restaurant staff
- Predict the review score using the text

Note that the original blog posts aimed to predict the next Michelin star. We will leave that as an extra challenge for you to try ...

The following tools and techniques will be demonstrated:

- How to setup a reproducible text pipeline in Python spaCy for text analysis;
- How to apply [topic modeling](http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf) as the primary tool to extract information from the review texts, to be combined and used in predictive modeling techniques to end up with our predictions
 

## Data preparation

Before we delve into the analytical side of things, we need some prepared textual data. As all true data scientists know, proper data preparation takes most of your time and is most decisive for the quality of the analysis results you end up with. Since preparing textual data is another cup of tea compared to preparing structured numeric or categorical data, and our goal is to show you how to do text analytics, we also want to show you how we cleaned and prepared the data we gathered. Therefore, in this notebook we start with the data dump with all reviews and explore and prepare this data in a number of steps:

![](https://github.com/jads-nl/public-lectures/blob/main/nlp/text-preprocessing-pipeline.png?raw=true)

In [1]:
import re

import pandas as pd
import pendulum


raw_reviews = pd.read_parquet("data/dutch-restaurant-reviews-per-year")
raw_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737058 entries, 0 to 737057
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   restoId              737058 non-null  int64   
 1   restoName            737058 non-null  object  
 2   tags                 713034 non-null  object  
 3   address              737058 non-null  object  
 4   scoreTotal           696162 non-null  float64 
 5   avgPrice             672448 non-null  object  
 6   numReviews           737058 non-null  int64   
 7   scoreFood            695968 non-null  float64 
 8   scoreService         695968 non-null  float64 
 9   scoreDecor           696074 non-null  float64 
 10  review_id            737058 non-null  float64 
 11  numreviews2          737058 non-null  float64 
 12  valueForPriceScore   593548 non-null  object  
 13  noiseLevelScore      593804 non-null  object  
 14  waitingTimeScore     593804 non-null  object  
 15  

Let's look at some `reviewText`s.

In [2]:
for review in raw_reviews.reviewText.head().to_list():
    print(review + "\n")

We komen al meer dan 8 jaar in dit restaurant en we genieten elke keer van het eten dat kies.Je krijgt ook goed advies wat je wel en niet kan combineren.De bediening is uitstekend en erg vriendelijk.Ik hoop hier nog vele jaren te gaan eten.

Een werkelijk prachtige ijssalon,blinkende uitstraling met zeer vriendelijke bediening. Het ijs werd in prachtige glazen opgediend en was van een meer dan uitstekende kwaliteit. De theepotjes waren sprookjesachtig en om verliefd op te worden!Wat een aanwinst! Wat een entourage en wat een verrukkelijk ijs!

Naast dat men hier heerlijk grieks eten heeft, is iedereen er altijd uitermate vriendelijk. Je wordt altijd snel geholpen met een lach. In de zomer is het terras een aanrader, in de winter zou ik alleen maar afhalen...
Vooral de stoofschotels zijn van een hoog grieks gehalte. Het vlees is mooi gaar en heeft precies de goede structuur. De overvloed aan bijgerechten maakt het helemaal af.

Via de Sweetdeal genoten van het 3 gangenkeuzemenu, we hebb

To get a better understanding of our data, let's check the most frequent, identical review texts:

In [3]:
raw_reviews.reviewText.value_counts(normalize=True).head(20).map(lambda x: '{:,.2f}%'.format(x*100))

- Recensie is momenteel in behandeling -    0.39%
Heerlijk gegeten!                           0.10%
Heerlijk gegeten                            0.08%
Heerlijk gegeten.                           0.04%
Top                                         0.03%
Lekker gegeten                              0.02%
Prima                                       0.02%
Top!                                        0.02%
Lekker gegeten!                             0.02%
Lekker gegeten.                             0.02%
Geen                                        0.01%
.                                           0.01%
Heerlijk gegeten!!                          0.01%
Prima restaurant                            0.01%
Goed                                        0.01%
Prima!                                      0.01%
Voor herhaling vatbaar                      0.01%
Nvt                                         0.01%
Heerlijk eten!                              0.01%
Weer heerlijk gegeten!                      0.01%


Ok, several things to solve here:

- Another 0,4% has the value '- Recensie is momenteel in behandeling -' (In English: The review is currently being processed) and therefore the actual review text is not published yet. Similar to empty reviews, we can delete these reviews.
- Several reviews seem very short and are not that helpful in trying to learn from the review text. Although this is very context dependent (when performing sentiment analysis, short reviews like 'Top!' (English: Top!), 'Prima' (Engish: Fine/OK) and 'Heerlijk gegeten' (En: Had a nice meal) might still have much value!) we will set a minimum length to reviews.

Note that that reviews containing no text have already been removed from the original dataset.

### Simple filtering on text length

In [4]:
def validate_review(review):
    if review == '- Recensie is momenteel in behandeling -' or len(review) < 4:
        return 0
    else:
        return 1
    

reviews = raw_reviews.loc[:, ['restoId', 'reviewerId', 'review_id', 'reviewerFame', 'reviewerNumReviews', 'reviewText']].copy()
reviews['is_valid'] = reviews.reviewText.apply(validate_review)
reviews[reviews.is_valid==0]['reviewText'].value_counts(normalize=True).head(10)

- Recensie is momenteel in behandeling -    0.802988
Top                                         0.059768
.                                           0.022136
Nvt                                         0.015495
-                                           0.011068
..                                          0.006641
nvt                                         0.004981
Kip                                         0.004981
Ok                                          0.004981
bof                                         0.003320
Name: reviewText, dtype: float64

So that looks OK, we can safely delete `is_valid == 0` reviews later. Let's do some more data prep.

### Parse localized datestrings with `pendulum`

In [5]:
pendulum.set_locale('nl')
pendulum.date(2021, 2, 12).format('D MMM YYYY')  # example

'12 feb. 2021'

In [6]:
def parse_date(date):
    return pendulum.from_format(date, fmt='D MMM YYYY', locale='nl')

reviews['reviewDate'] = raw_reviews.reviewDate.apply(parse_date).dt.date

In [7]:
reviews.reviewDate.head()

0    2012-09-19
1    2012-07-12
2    2012-11-29
3    2012-12-13
4    2012-10-19
Name: reviewDate, dtype: object

### Format numerical columns

In [8]:
# avgPrice has whitespace and euro character
def clean_price(string):
    if string:
        return string.split(" ")[-1]
    else:
        return None


reviews["avgPrice"] = raw_reviews["avgPrice"].apply(clean_price)

In [9]:
# turn categorical columns into ordinal values, lower is better
# note to Dutch audience: do you think the ordinal order is sensible and correct?
map_scores = {
    "waitingTimeScore": {
        "Hoog tempo": 1,
        "Kort": 2,
        "Redelijk": 3,
        "Kan beter": 4,
        "Lang": 5,
    },
    "valueForPriceScore": {
        "Erg gunstig": 1,
        "Gunstig": 2,
        "Redelijk": 3,
        "Precies goed": 4,
        "Kan beter": 5,
    },
    "noiseLevelScore": {
        "Erg rustig": 1,
        "Rustig": 2,
        "Precies goed": 3,
        "Rumoerig": 4,
    },
}

for col in map_scores.keys():
    reviews[col] = (
        raw_reviews[col].apply(lambda x: map_scores[col].get(x, None)).astype("Int64")
    )

In [10]:
# numerical columns have comma as decimal seperator --> cast to floats
numerical_cols = [
    "scoreFood",
    "scoreService",
    "scoreDecor",
    "reviewScoreOverall",
    "scoreTotal",
]
for col in numerical_cols:
    reviews[col] = pd.to_numeric(raw_reviews[col])

In [11]:
reviews.head()

,restoId,reviewerId,review_id,reviewerFame,reviewerNumReviews,reviewText,is_valid,reviewDate,avgPrice,waitingTimeScore,valueForPriceScore,noiseLevelScore,scoreFood,scoreService,scoreDecor,reviewScoreOverall,scoreTotal
0,236127,111373143.0,20.0,Fijnproever,4.0,We komen al meer dan 8 jaar in dit restaurant ...,1,2012-09-19,35,<NA>,<NA>,<NA>,8.6,8.4,7.2,8.5,8.4
1,246631,111355027.0,11.0,Meesterproever,21.0,"Een werkelijk prachtige ijssalon,blinkende uit...",1,2012-07-12,None,<NA>,<NA>,<NA>,8.2,7.6,8.0,10.0,8.0
2,243427,112961389.0,3.0,Expertproever,9.0,"Naast dat men hier heerlijk grieks eten heeft,...",1,2012-11-29,None,<NA>,<NA>,<NA>,NaN,NaN,NaN,8.0,NaN
3,234077,111347867.0,107.0,Meesterproever,97.0,Via de Sweetdeal genoten van het 3 gangenkeuze...,1,2012-12-13,45,<NA>,<NA>,<NA>,8.0,8.0,7.6,7.0,7.9
4,240845,112167929.0,14.0,Meesterproever,40.0,Vakantieveiling is een leuk ding om restaurant...,1,2012-10-19,43,<NA>,<NA>,<NA>,7.3,7.6,7.4,8.5,7.4


### Exercise: perform exploratory data analysis

Prior to diving into NLP with spaCy, perform a EDA to explore possible correlations:
- reviewer type vs. given scores
- length of reviews vs. scores
- value-for-money vs. scores

Learning objective:
- Lest you forget to always do a short EDA, before getting lost in details ...

## Getting started with spaCy

To develop reproducible pipelines, we will follow the recommended workflow from spaCy.

![](https://spacy.io/pipeline-fde48da9b43661abcdf62ab70a546d71.svg)

When you call `nlp` on a text, spaCy first tokenizes the text to produce a `Doc` object. The `Doc` is then processed in several different steps – this is also referred to as the __processing pipeline__. The pipeline used by the [trained pipelines](https://spacy.io/models) typically include a tagger, a lemmatizer, a parser and an entity recognizer. Each pipeline component returns the processed `Doc`, which is then passed on to the next component.

The tokenizer is a “special” component and isn’t part of the regular pipeline. It also doesn’t show up in `nlp.pipe_names`. The reason is that there can only really be one tokenizer, and while all other pipeline components take a `Doc` and return it, the tokenizer takes a __string of text__ and turns it into a `Doc`. You can still customize the tokenizer, though. `nlp.tokenizer` is writable, so you can either create your own [`Tokenizer` class from scratch](https://spacy.io/usage/linguistic-features#native-tokenizers), or even replace it with an [entirely custom function](https://spacy.io/usage/linguistic-features#custom-tokenizer).

We will use the large Dutch model which is 546 MB in size. The download command needs to be run once on your system. You may want to restart your Jupyter Notebook kernel to ensure spaCy is loaded properly with the newly downloaded model.

In [12]:
# !python -m spacy download nl_core_news_lg

In [13]:
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy


nlp = spacy.load("nl_core_news_lg")

We will show different options of tokenizing, see [this blog by Dataquest](https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/) for more details.

### Documents in spaCy

In [14]:
# create spaCy doc from one of the reviews
example_doc = nlp(reviews.reviewText[100])

# Doc object has 51 attributes and methods
print(f"Methods and attributes of spaCy Doc object: {[attr for attr in dir(example_doc) if not '__' in attr]}")

Methods and attributes of spaCy Doc object: ['_', '_bulk_merge', '_py_tokens', '_realloc', '_vector', '_vector_norm', 'cats', 'char_span', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_disk', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'merge', 'noun_chunks', 'noun_chunks_iterator', 'print_tree', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_extension', 'similarity', 'tensor', 'text', 'text_with_ws', 'to_array', 'to_bytes', 'to_disk', 'to_json', 'to_utf8_array', 'user_data', 'user_hooks', 'user_span_hooks', 'user_token_hooks', 'vector', 'vector_norm', 'vocab']


In [15]:
# visualize named entities
spacy.displacy.render(example_doc, style='ent')

In [16]:
# visualize dependencies
spacy.displacy.render(example_doc, style='dep')

### Tokens in spaCy

In [17]:
# example_doc holds all tokens
print(f"Length of example doc: {len(example_doc)}")

# which you can access as a list
print(f"First token example doc: {example_doc[0]}")

# each token is a spaCy Token object with 90 methods and attributes
token_attrs = [attr for attr in dir(example_doc[0]) if not '__' in attr]
print(f"Each token is a {type(example_doc[0])} with 90 attributes:")
print(token_attrs)

Length of example doc: 84
First token example doc: We
Each token is a <class 'spacy.tokens.token.Token'> with 90 attributes:
['_', 'ancestors', 'check_flag', 'children', 'cluster', 'conjuncts', 'dep', 'dep_', 'doc', 'ent_id', 'ent_id_', 'ent_iob', 'ent_iob_', 'ent_kb_id', 'ent_kb_id_', 'ent_type', 'ent_type_', 'get_extension', 'has_extension', 'has_vector', 'head', 'i', 'idx', 'is_alpha', 'is_ancestor', 'is_ascii', 'is_bracket', 'is_currency', 'is_digit', 'is_left_punct', 'is_lower', 'is_oov', 'is_punct', 'is_quote', 'is_right_punct', 'is_sent_end', 'is_sent_start', 'is_space', 'is_stop', 'is_title', 'is_upper', 'lang', 'lang_', 'left_edge', 'lefts', 'lemma', 'lemma_', 'lex_id', 'like_email', 'like_num', 'like_url', 'lower', 'lower_', 'morph', 'n_lefts', 'n_rights', 'nbor', 'norm', 'norm_', 'orth', 'orth_', 'pos', 'pos_', 'prefix', 'prefix_', 'prob', 'rank', 'remove_extension', 'right_edge', 'rights', 'sent', 'sent_start', 'sentiment', 'set_extension', 'shape', 'shape_', 'similarity', 

In [18]:
# let's view token attributes we want to use
pd.DataFrame(
    [
        (
            token.text,
            token.lower_,
            token.lemma_,
            token.shape_,
            token.is_alpha,
            token.is_stop,
            token.is_punct,
            token.tag_,
        )
        for token in example_doc
    ],
    columns=[
        "text",
        "lower_",
        "lemma_",
        "shape_",
        "is_alpha",
        "is_stop",
        "is_punct",
        "tag_",
    ],
)

,text,lower_,lemma_,shape_,is_alpha,is_stop,is_punct,tag_
0,We,we,we,Xx,True,True,False,VNW|pers|pron|nomin|red|1|mv__Case=Nom|Person=...
1,kozen,kozen,kiezen,xxxx,True,False,False,WW|pv|verl|mv__Number=Plur|Tense=Past|VerbForm...
2,voor,voor,voor,xxxx,True,True,False,VZ|init
3,dit,dit,dit,xxx,True,True,False,VNW|aanw|det|stan|prenom|zonder|evon
4,restaurant,restaurant,restaurant,xxxx,True,False,False,N|soort|ev|basis|onz|stan__Gender=Neut|Number=...
...,...,...,...,...,...,...,...,...
79,in,in,in,xx,True,True,False,VZ|init
80,positieve,positieve,positief,xxxx,True,False,False,ADJ|prenom|basis|met-e|stan__Degree=Pos
81,zin,zin,zin,xxx,True,False,False,N|soort|ev|basis|zijd|stan__Gender=Com|Number=...
82,),),),),False,False,True,LET


### Simple tokenizer

In [19]:
def tokenize_simple(text):
    """Tokenizer returning lowercase tokens with no stop words, no punctuation and no words with encoding errors"""
    doc = nlp(text)
    return [token.lower_ for token in doc if not (token.is_stop or token.is_punct or ("\\" in token.lower_))]

tokenize_simple(reviews.reviewText[100])

['kozen',
 'restaurant',
 'vieren',
 'absoluut',
 'gelukt',
 '4-gangen',
 'verrassingsmenu',
 'voorafgegaan',
 'twee',
 'amuses',
 'vóór',
 'dessert',
 'kregen',
 'prédessert',
 'wijnarrangement',
 'beroerd',
 'geschonken',
 'kortom',
 'weet',
 'klant',
 'watten',
 'leggen',
 'voelt',
 'echt',
 'gast',
 'eten',
 'heerlijk',
 'bood',
 'randstedelijke',
 'ogen',
 'absurde',
 'prijs-kwaliteitsverhouding',
 'positieve',
 'zin']

### Using lemmas as tokens

In [20]:
def tokenize_lemma(text):
    """Tokenizer returning lemmas with no stop words, no punctuation and no words with encoding errors"""
    doc = nlp(text)
    return [token.lemma_ for token in doc if not (token.is_stop or token.is_punct or ("\\" in token.lower_))]

tokenize_lemma(reviews.reviewText[100])

['kiezen',
 'restaurant',
 'vieren',
 'absoluut',
 'lukken',
 '4-gang',
 'verrassingsmenu',
 'voorafgaan',
 'twee',
 'amuse',
 'vóór',
 'dessert',
 'krijgen',
 'prédessert',
 'wijnarrangement',
 'beroerd',
 'schenken',
 'kortom',
 'weten',
 'klant',
 'watten',
 'leggen',
 'voelen',
 'echt',
 'ga',
 'eten',
 'heerlijk',
 'bieden',
 'randstedelijk',
 'oog',
 'absurd',
 'prijs-kwaliteitsverhouding',
 'positief',
 'zin']

### Exercises

Since it takes quite some computing power to process all reviews, it is suggested to do the exercises just for a given year.

#### Remove accents in the tokens such that `vóór` is tokenized as `voor` 

#### Explore association between overall review score and length of text

#### Extract ADJ-NOUN bigrams from the text

Imagine you are advising the owner of a restaurant. We wants to mine the reviews to find specific improvement points. Assuming that ADJECTIVE - NOUN bigrams can provide useful input, for example 'oude servies', 'lange wachttijd', your task is to:
- Extract ADJ-NOUN bigrams for each review
- Find the most interesting bigrams for positive and negative reviews per restaurant

## Document-Term Matrix


### Unigrams and bag-of-words
Use `sklearn.feature_extraction.text.CountVectorizer` to create the DTM. Since this is a large and sparse matrix, this data in a numpy array instead of a pandas dataframe. We will use the spaCy's `Defaults.stop_words`. Since creating the DTM for all reviews takes a long time, we will test it first with just 1,000 records

In [21]:
%%time
count_vectorizer = CountVectorizer(tokenizer=tokenize_simple, stop_words=nlp.Defaults.stop_words, ngram_range=(1,1))
count_vectorizer.fit_transform(reviews.reviewText[:1000])

/home/dkapitan/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['n', 't'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CPU times: user 21.5 s, sys: 12 ms, total: 21.5 s
Wall time: 21.5 s


<1000x8281 sparse matrix of type '<class 'numpy.int64'>'
	with 37103 stored elements in Compressed Sparse Row format>

That's an interesting warning. Let's look at the stopwords that we have used.

In [22]:
print(list(nlp.Defaults.stop_words))

['zijn', 'voordien', 'moeten', 'effe', 'waarom', 'nabij', 'alle', 'de', 'zoveel', 'verre', 'onszelf', 'aldus', 'mogelijk', 'wij', 'zodra', 'door', 'omhoog', 'zelfs', 'ons', 'anderen', 'veel', 'voor', 'dezen', 'ooit', 'weer', 'eerder', 'altijd', 'zeker', 'des', 'kunt', 'toch', 'indien', 'voorop', 'zou', 'vanuit', 'omver', 'dien', 'wordt', 'inmiddels', 'boven', 'ook', 'nogal', 'als', 'achterna', 'zei', 'haar', 'zijne', 'jijzelf', 'publ', 'verder', 'zult', 'gewoon', 'zal', 'enige', 'liet', 'hier', 'zullen', 'beneden', 'ikke', 'aan', 'zonder', 'uw', 'ikzelf', 'bijvoorbeeld', 'na', 'u', 'vgl', 'wier', 'voorbij', 'weinige', 'reeds', 'al', 'hierbeneden', 'afgelopen', 'deze', 'hebt', 'welke', 'mijn', '‘t', 'men', 'ben', 'toe', 'hun', 'konden', 'evenwel', 'nu', 'klaar', 'ja', 'geen', 'hen', 'eigen', 'alhoewel', 'werden', 'voorts', 'alles', 'hij', 'dit', 'pas', 'maar', 'zoals', 'onder', 'wie', 'omtrent', 'vanaf', 'onze', 'wil', 'krachtens', 'rondom', 'nooit', 'doet', 'nr', 'thans', 'volgens', 'f

So what happens here is:
- spaCy tokenizes, for example, the stop word `zo'n` into `[zo, ', n]`
- since we have chosen to remove punctuation we end up with `[zo, n]` as the tokens of the original stop word
- `zo` was already in the list of stop words, but `n` isn't which is what the warning is about

Adding those two fixes the warning, so now we can move on to actually calculate the most frequent words.

In [23]:
%%time
stop_words = nlp.Defaults.stop_words.update(['n', 't'])
count_vectorizer = CountVectorizer(tokenizer=tokenize_simple, stop_words=stop_words, ngram_range=(1,1))
bow = count_vectorizer.fit_transform(reviews.reviewText[:1000]) # just 1,000 reviews for demonstration

CPU times: user 12.2 s, sys: 234 µs, total: 12.2 s
Wall time: 12.2 s


Let's inspect the most frequent unigrams.

In [24]:
unigrams = pd.Series(bow.sum(axis=0).A1.transpose().ravel(), index=count_vectorizer.get_feature_names())
unigrams.sort_values(ascending=False)

\n                945
eten              626
bediening         573
goed              546
erg               450
                 ... 
onderhoud           1
onderhand           1
ondergetekende      1
eetgewoontes        1
5,50                1
Length: 8281, dtype: int64

### TF-IDF
In the same vein, we could use the `TfidfVectorizer`. While we are at it, let's include bigrams, too.

In [25]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize_simple, stop_words=stop_words, ngram_range=(1,2))
tfidf = tfidf_vectorizer.fit_transform(reviews.reviewText[:1000])

Let's see which words have the highest TF-IDF per document.

In [26]:
rare_bigrams = pd.Series([tfidf_vectorizer.get_feature_names()[i] for i in tfidf.A.argmax(axis=1)])
rare_bigrams.head(40)

0                    8 jaar
1                 prachtige
2                    grieks
3                         4
4                uur rijden
5                        \n
6                       uur
7                  iegewies
8                2 desserts
9                   broodje
10             aandoet oude
11                   6 euro
12                   gangen
13                      wok
14               \n maarrrr
15                     ribs
16          groupon voucher
17                    17,95
18               broodjemet
19                    bezig
20                 2-jarige
21                     soep
22             correct soms
23                    2 ker
24          achterin lounge
25    bediening fantastisch
26            attent genoeg
27                     ruim
28                    soort
29                  izakaya
30                      bar
31           kabouterhappen
32                    rijst
33             attente soms
34                      top
35              groe

Interesting to read a review mentioning 'kabouterhappen':

In [27]:
print(reviews.reviewText[31])

Ik had me erg verheugd op ons etentje bij Izakaya. De aankleding  van het restaurant ziet er veelbelovend uit. Ik dacht eindelijk een 'place to be' in de Pijp. Bij de reservering kreeg ik al een naar gevoel; de tafel was beschikbaar van 19.00 tot 21.00 uur. De eerste indruk bij binnenkomst was prima, aardige bediening, leuke tafel. Wel veel trendy dertigers en typisch Oud-Zuid publiek.     We hebben meerdere ronde hapjes bestelt, sommige echt lekker, andere middelmatig. Wat met name teleurstelde, was de hoeveelheid. We zijn geen grote eters, maar dit waren echt kabouterhappen. De grootste tegenvaller kwam achteraf; de rekening. Duurbetaalde kabouterhappen. We waren met 4 mensen ruim €280,- kwijt. In deze prijsklasse ken ik veel betere restaurants in Amsterdam. Als Izakaya de porties een beetje groter maakt en de prijzen een beetje kleiner, is er hoop.


We will use the document-term matrix, with either simple counts or with TF-IDF, later on in our classification pipeline.

## Topic modeling

In [38]:
# https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html?highlight=topic%20modeling
# https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn


pyLDAvis.enable_notebook()

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

tfidf_vectorizer2 = TfidfVectorizer(
    max_df=0.95,
    min_df=2,
    max_features=n_features,
    tokenizer=tokenize_simple,
    stop_words=stop_words,
    strip_accents='unicode',
    ngram_range=(1, 2),
)
dtm_tfidf = tfidf_vectorizer2.fit_transform(reviews.reviewText[:n_samples])

/home/dkapitan/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
lda_tfidf = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=5,
    learning_method="online",
    learning_offset=50.0,
    random_state=0,
).fit(dtm_tfidf)

/home/dkapitan/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer2)

/home/dkapitan/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.062451  0.000352       1        1  83.935547
0      0.016710 -0.022568       2        1   1.867840
4      0.020517  0.018810       3        1   1.845631
3      0.005328  0.000855       4        1   1.774866
9     -0.001274  0.000457       5        1   1.762687
7      0.002708  0.000280       6        1   1.762686
5      0.004014  0.000434       7        1   1.762686
1      0.004235  0.000443       8        1   1.762686
6      0.005161  0.000456       9        1   1.762686
8      0.005054  0.000483      10        1   1.762686, topic_info=           Term        Freq       Total Category  logprob  loglift
0            \n  109.000000  109.000000  Default  30.0000  30.0000
232         erg   64.000000   64.000000  Default  29.0000  29.0000
89    bediening   71.000000   71.000000  Default  28.0000  28.0000
539      lekker   65.000000   65.000000  Default  27.0000  27.0000
363        goed   78.000000   78.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
301     gegeten    0.181890   61.226944  Topic10  -6.7981  -1.7806
89    bediening    0.181251   71.164634  Topic10  -6.8016  -1.9345
548       leuke    0.179204   26.108121  Topic10  -6.8130  -0.9431
724  restaurant    0.180477   60.018818  Topic10  -6.8059  -1.7685
406    heerlijk    0.180135   57.679875  Topic10  -6.8078  -1.7306

[765 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
0         1  0.984590               \n
3         1  0.541900         \n decor
4         1  0.563806       \n dessert
5         1  0.709998           \n erg
7         1  0.633120  \n hoofdgerecht
...     ...       ...              ...
987       1  0.910342           zitten
988       1  0.826307             zo'n
992       1  0.650213             zoet
993       1  0.725174            zoete
999       1  0.892764                €

[387 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 5, 4, 10, 8, 6, 2, 7, 9])

## Classification challenge

As a final exercise, combine all the different techniques we have covered so far to build a model that predicts the review score using the text. Use `TransformerMixin` to encapsulate all the functions into one sklearn pipeline.

In [ ]:
class tokenize(TransformerMixin):
    def transform(self, X, **transform_params):
        return [tokenize_simple(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

In [ ]:
# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])